In [26]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

from data_reader import read_data
from data_preprocessing import preprocess_data
from feature_extractor import extract_features

%matplotlib inline

In [4]:
data = read_data()
data = preprocess_data(data, limit=10000)
data.head()

,id,Title,Uri,PublicationDateTimeUTC,ProcedureDisplayName,Amount,RubPrice,CurrencyCode,StatusDisplayName,StatusCode,SuppliersCount,IsWinner,Inn,Kpp,Name,Ogrn,ResultClass,Ogrn1
0,0100600000117000008,услуги по капитальному ремонту,https://zakupki.kontur.ru/0100600000117000008,2017-07-27,Закупка у единственного поставщика (подрядчика...,352000.00,352000.00,RUB,Размещение отменено,3.0,0.0,0.0,4720008346,NaN,МЕСТНАЯ АДМИНИСТРАЦИЯ МУНИЦИПАЛЬНОГО ОБРАЗОВАН...,1024702184451,2.0,47
1,0101100000116000162,Масло сладко-сливочное несоленое Крестьянское,https://zakupki.kontur.ru/0101100000116000162,2016-10-24,Электронный аукцион,3855260.00,3855260.00,RUB,Размещение отменено,3.0,0.0,0.0,0275006455,NaN,УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ИСПОЛНЕНИЯ НАКАЗ...,1020202776714,2.0,2
2,0101100000117000153,Индивидуальный рацион питания для повседневной...,https://zakupki.kontur.ru/0101100000117000153,2017-09-25,Электронный аукцион,3600000.00,3600000.00,RUB,Размещение отменено,3.0,0.0,0.0,0275006455,NaN,УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ИСПОЛНЕНИЯ НАКАЗ...,1020202776714,2.0,2
3,0101100000117000154,Консервы мясные,https://zakupki.kontur.ru/0101100000117000154,2017-09-25,Электронный аукцион,15400000.00,15400000.00,RUB,Размещение отменено,3.0,0.0,0.0,0275006455,NaN,УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ИСПОЛНЕНИЯ НАКАЗ...,1020202776714,2.0,2
4,0101100000416000025,Право заключения государственного контракта на...,https://zakupki.kontur.ru/0101100000416000025,2016-05-30,Открытый конкурс,79196.94,79196.94,RUB,Размещение отменено,3.0,0.0,0.0,0278103383,NaN,УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ПО НАДЗОРУ В СФЕ...,1040204605154,2.0,2


In [5]:
# def GetBalancedSample(data, count):
#     cancel_count = min(count // 3, data[data.ResultClass == 2].shape[0])
#     success_count = min((count - cancel_count) // 2, data[data.ResultClass == 1].shape[0])
#     unsuccess_count = count - cancel_count - success_count
#     balanced_sample = pd.concat([data[data.ResultClass == 2][:cancel_count],
#                                  data[data.ResultClass == 1][:success_count],
#                                  data[data.ResultClass == 0][:unsuccess_count]])
#     print(balanced_sample.groupby('ResultClass').size())
#     return balanced_sample

# GetBalancedSample(data, 10000)

In [6]:
data.groupby('ResultClass').size()

ResultClass
0.0    3334
1.0    3333
2.0    3333
dtype: int64

In [7]:
data.corr()

,Amount,RubPrice,StatusCode,SuppliersCount,IsWinner,ResultClass,Ogrn1
Amount,1.000000,1.000000,0.049623,-0.007320,-0.008976,0.049587,0.002860
RubPrice,1.000000,1.000000,0.049623,-0.007320,-0.008976,0.049587,0.002860
StatusCode,0.049623,0.049623,1.000000,-0.397528,-0.456661,0.866025,0.476021
SuppliersCount,-0.007320,-0.007320,-0.397528,1.000000,0.811654,-0.006131,-0.152609
IsWinner,-0.008976,-0.008976,-0.456661,0.811654,1.000000,0.033552,-0.164021
ResultClass,0.049587,0.049587,0.866025,-0.006131,0.033552,1.000000,0.438440
Ogrn1,0.002860,0.002860,0.476021,-0.152609,-0.164021,0.438440,1.000000


In [8]:
data_head = data
features = extract_features(data_head)[0]

oh CurrencyCode
oh ProcedureDisplayName
oh Ogrn
ng Title
ng ProcedureDisplayName
ng OrgName


In [9]:
features = pd.DataFrame(features)

In [10]:
features.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(features.drop('ResultClass', axis=1).values,
                            features.ResultClass.values,
                            test_size=0.3,
                            random_state=123456)

In [11]:
# features = pd.DataFrame(features)
# X_train = features.drop(['ResultClass'], 1)
# y_train = features.ResultClass

In [12]:
# rf = RF(n_estimators=10, n_jobs=-1)
# rf = rf.fit(x_train, y_train)
# np.mean(cross_val_score(rf, x_train, y_train))

In [13]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, label=y_test, reference=lgb_train, free_raw_data=False)

In [14]:
params = {
        'objective': 'multiclass',
        'metric': 'multi_error',
        'num_classes': 3,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 123456,
        'bagging_fraction': 0.9,
        'bagging_freq': 1,
        'bagging_seed': 123456,
        'max_depth': 8,
        'learning_rate': 0.1,
        'min_data_in_leaf': 11,
        'num_iteration': 100,
        'num_threads': 2
    }

In [15]:
gbm = lgb.train(params,
                    lgb_train,
                    valid_sets=[lgb_train, lgb_eval])  # eval training data
#                     feature_name=features.drop('ResultClass', axis=1).columns.values)

gbm.save_model(f'result.txt')

C:\anaconda3\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's multi_error: 0.0791429	valid_1's multi_error: 0.105667
[2]	training's multi_error: 0.073	valid_1's multi_error: 0.102667
[3]	training's multi_error: 0.0688571	valid_1's multi_error: 0.1
[4]	training's multi_error: 0.0714286	valid_1's multi_error: 0.0976667
[5]	training's multi_error: 0.0698571	valid_1's multi_error: 0.102
[6]	training's multi_error: 0.064	valid_1's multi_error: 0.098
[7]	training's multi_error: 0.0612857	valid_1's multi_error: 0.0943333
[8]	training's multi_error: 0.0622857	valid_1's multi_error: 0.0966667
[9]	training's multi_error: 0.0591429	valid_1's multi_error: 0.0926667
[10]	training's multi_error: 0.0574286	valid_1's multi_error: 0.094
[11]	training's multi_error: 0.0555714	valid_1's multi_error: 0.0916667
[12]	training's multi_error: 0.0555714	valid_1's multi_error: 0.0906667
[13]	training's multi_error: 0.0545714	valid_1's multi_error: 0.0936667
[14]	training's multi_error: 0.0541429	valid_1's multi_error: 0.091
[15]	training's multi_error: 0.05

In [16]:
num_round = 12
lgb.cv(params, lgb_train, num_round, nfold=5)

{'multi_error-mean': [0.094571897770292998,
  0.093142202653413539,
  0.092285466503265245,
  0.086426585964267749,
  0.087856687788141791,
  0.087712504477491487,
  0.0881425037506047,
  0.086712911621492139,
  0.085570461622635624,
  0.084427807833275775,
  0.084284135528405163,
  0.083999746099465783],
 'multi_error-stdv': [0.0078525462477377315,
  0.0055321708320201935,
  0.0037566893193274587,
  0.0054637108036542066,
  0.0029483854732306608,
  0.0033749924479041992,
  0.0021799606942215759,
  0.0030103491501607006,
  0.0031491924351767786,
  0.003122618610625163,
  0.0044866421646252855,
  0.0040513004360489286]}

In [19]:
# fit model no training data
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
np.mean(cross_val_score(xgb, X_train, y_train, cv=5))

0.91642627860332282

In [20]:
from sklearn.ensemble import RandomForestClassifier as RF

rf = RF(n_estimators=200, max_features='auto', n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
def get_metric(X_test, y_test, model):
    y_pred = None
    if "predict_proba" in dir(model):
        y_pred = model.predict_proba(X_test)
    else:
        y_pred = model.predict(X_test)
    test = pd.DataFrame(y_test, columns=['test'])
    test['prediction'] = [np.argmax(x) for x in y_pred]
    print(classification_report(y_test, test['prediction'].values))
    test['is_same'] = test.apply(lambda row: int(row[0] == row[1]), axis=1)
    f = {'is_same': ['sum', 'size', 'mean']}
    groups = test.groupby('test').agg(f)
    print(sum(test['is_same']) / test.shape[0])
    return groups['is_same'].sort_values('mean', ascending=False)

In [22]:
get_metric(X_test, y_test, gbm)

             precision    recall  f1-score   support

        0.0       0.90      0.92      0.91       988
        1.0       0.86      0.90      0.88       987
        2.0       0.99      0.94      0.96      1025

avg / total       0.92      0.92      0.92      3000

0.918666666667


,sum,size,mean
test,,,
2.0,959,1025,0.935610
0.0,908,988,0.919028
1.0,889,987,0.900709


In [23]:
get_metric(X_test, y_test, xgb)

             precision    recall  f1-score   support

        0.0       0.90      0.90      0.90       988
        1.0       0.84      0.90      0.87       987
        2.0       0.99      0.93      0.96      1025

avg / total       0.91      0.91      0.91      3000

0.908333333333


,sum,size,mean
test,,,
2.0,952,1025,0.928780
0.0,887,988,0.897773
1.0,886,987,0.897670


In [24]:
get_metric(X_test, y_test, rf)

             precision    recall  f1-score   support

        0.0       0.93      0.91      0.92       988
        1.0       0.85      0.92      0.88       987
        2.0       0.98      0.93      0.96      1025

avg / total       0.92      0.92      0.92      3000

0.918666666667


,sum,size,mean
test,,,
2.0,953,1025,0.929756
1.0,908,987,0.919959
0.0,895,988,0.905870


In [ ]:
model = CatBoostClassifier(iterations=100, depth=10, learning_rate=.1, loss_function='MultiClass', classes_count=3)
model.fit(X_train, y_train)

0:	learn: -0.9678558	total: 389ms	remaining: 38.5s
1:	learn: -0.8593531	total: 814ms	remaining: 39.9s
2:	learn: -0.7774173	total: 1.27s	remaining: 41.1s
3:	learn: -0.7053065	total: 1.69s	remaining: 40.6s
4:	learn: -0.6471666	total: 2.11s	remaining: 40.2s
5:	learn: -0.6016406	total: 2.55s	remaining: 40s
6:	learn: -0.5625560	total: 3s	remaining: 39.8s
7:	learn: -0.5295536	total: 3.36s	remaining: 38.7s
8:	learn: -0.4988725	total: 3.81s	remaining: 38.6s
9:	learn: -0.4693378	total: 4.21s	remaining: 37.9s
10:	learn: -0.4431673	total: 4.65s	remaining: 37.6s
11:	learn: -0.4245586	total: 4.8s	remaining: 35.2s
12:	learn: -0.4049361	total: 5.25s	remaining: 35.1s
13:	learn: -0.3867700	total: 5.71s	remaining: 35.1s
14:	learn: -0.3735764	total: 6.14s	remaining: 34.8s
15:	learn: -0.3588691	total: 6.54s	remaining: 34.3s
16:	learn: -0.3467811	total: 6.94s	remaining: 33.9s
17:	learn: -0.3371953	total: 7.13s	remaining: 32.5s
18:	learn: -0.3267381	total: 7.56s	remaining: 32.2s
19:	learn: -0.3169927	total:

In [ ]:
get_metric(X_test, y_test, model)